In [1]:
import torch
import torch.nn as nn
import numpy as np
import sys
import os
import random
import matplotlib.pyplot as plt

src_path = os.path.abspath(os.path.join(os.getcwd(), 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)
    
from utils import MIMONetDataset, DeepONetDataset, ChannelScaler
from mimonet import MIMONet

In [2]:
# check if GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [3]:
# set working directory
working_dir = "/projects/bcnx/kazumak2/MIMONet/LDC/"
data_dir = os.path.join(working_dir, "data")

## load datasets

### Load sharing parameters/dataset

In [16]:
# trunk dataset
trunk_input = np.load(os.path.join(data_dir, "share/coords.npy"))

### Training data

In [17]:
# training data
train_branch = np.load(os.path.join(data_dir, "training/train_branch_input.npy"))

# [samples, channel, gridpoints]
train_target = np.load(os.path.join(data_dir, "training/train_target.npy"))


print("train_branch shape:", train_branch.shape)
print("train_target shape:", train_target.shape)

train_branch shape: (3949, 90)
train_target shape: (3949, 4225, 3)


In [18]:
# scaling the train_branch data [min-max scaling]
b_max = np.max(train_branch)
b_min = np.min(train_branch)

train_branch = 2 * (train_branch - b_min) / (b_max - b_min) - 1

print('branch input min:', b_min)
print('branch input max:', b_max)

branch input min: -0.953423
branch input max: 13.645064


### Test data

In [20]:
test_branch = np.load(os.path.join(data_dir, "test/test_branch_input.npy"))

test_target = np.load(os.path.join(data_dir, "test/test_target.npy"))

print("test_branch shape:", test_branch.shape)
print("test_target shape:", test_target.shape)

# scaling the test_branch data [min-max scaling]
test_branch = 2 * (test_branch - b_min) / (b_max - b_min) - 1

test_branch shape: (988, 90)
test_target shape: (988, 4225, 3)


### Scaling the target data

In [21]:
# scaling the target data
'''  
note: reverse the scaling for the target data
train_target = scaler.inverse_transform(train_target_scaled)
test_target = scaler.inverse_transform(test_target_scaled)
'''
scaler = ChannelScaler(method='minmax', feature_range=(-1, 1))
scaler.fit(train_target)
train_target_scaled = scaler.transform(train_target)
test_target_scaled = scaler.transform(test_target)


## Torch Dataset and DataLoader

In [22]:
# test dataset and dataloader
test_dataset = MIMONetDataset(
    [test_branch],  # branch_data_list
    trunk_input,                     # trunk_data
    test_target_scaled               # target_data
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,  # set to 1 for testing
    shuffle=False,
    num_workers=0
)

In [23]:
train_dataset = MIMONetDataset(
    [train_branch],  # branch_data_list
    trunk_input,                       # trunk_data
    train_target_scaled                # target_data
)

## MIMONet Model

In [24]:
# Architecture parameters
dim = 256
branch_input_dim1 = 90
branch_input_dim2 = 2
trunk_input_dim = 2

# Define MIONet instance (no Fourier, no final linear)
model = MIMONet(
    branch_arch_list=[
        [branch_input_dim1, 512, 512, 512, dim]
    ],
    trunk_arch=[trunk_input_dim, 256, 256, 256, dim],
    num_outputs=3, 
    activation_fn=nn.ReLU,
    merge_type='mul'  # or 'sum'
)

model = model.to(device)

# Print parameter count
num_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {num_params:,}")

Total number of parameters: 1,032,963
